In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'CS231n/Project/cifar-10'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME


Mounted at /content/drive
/content/drive/My Drive/CS231n/Project/cifar-10/datasets
/content/drive/My Drive/CS231n/Project/cifar-10


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

using device: cuda


In [ ]:
NUM_TRAIN = 49000

transform_train = T.Compose([
                       T.RandomCrop(32,padding=2),
                       T.RandomHorizontalFlip(),
                       T.ToTensor(),
                       T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = T.Compose([
                       T.ToTensor(),
                       T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

cifar10_train = dset.CIFAR10('./datasets', train=True, download=True, transform=transform_train)

loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./datasets', train=True, download=True, transform=transform_train)

loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, transform=transform_test)

loader_test = DataLoader(cifar10_test, batch_size=64)




Extracting ./datasets/cifar-10-python.tar.gz to ./datasets
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
      print('Checking accuracy on val set')
    else:
      print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
      for x,y in loader:
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)
        scores = model(x)
        _, preds = scores.max(1)
        num_correct += (preds==y).sum()
        num_samples += preds.size(0)
      acc = float(num_correct)/num_samples
      print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
def train(model, optimizer, epochs=1):
    #print('\nEpoch: %d' % epochs)
    model = model.to(device=device)
    for e in range(epochs):
      print('\nEpoch: %d' % e)
      for t, (x,y) in enumerate(loader_train):
        model.train()
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        scores = model(x)
        loss = F.cross_entropy(scores, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,stride=1, padding=1, bias=False)
        #self.bn2 = nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(0.2)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.conv2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        out = self.dropout(out)
        #out = self.bn1(out)
        return out

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        #self.bn1 = nn.BatchNorm2d(64) 
        self.dropout = nn.Dropout(0.2)
        self.layer1 = BasicBlock(64, 64, 1)
        self.layer2 = BasicBlock(64,128,2)
        self.layer3 = BasicBlock(128, 256,2)
        self.layer4 = BasicBlock(256, 512, 2)
        self.linear = nn.Linear(512, num_classes)
        
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.layer1(out)
        #out = self.dropout(out)
        out = self.layer2(out)
        #out = self.dropout(out)
        out = self.layer3(out)
        #out = self.dropout(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [ ]:
model = ConvNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model, optimizer, 1)


Epoch: 0
Iteration 0, loss = 2.3064
Checking accuracy on val set
Got 119 / 1000 correct (11.90)

Iteration 100, loss = 1.9756
Checking accuracy on val set
Got 223 / 1000 correct (22.30)

Iteration 200, loss = 1.8265
Checking accuracy on val set
Got 339 / 1000 correct (33.90)

Iteration 300, loss = 1.7690
Checking accuracy on val set
Got 386 / 1000 correct (38.60)

Iteration 400, loss = 1.5773
Checking accuracy on val set
Got 404 / 1000 correct (40.40)

Iteration 500, loss = 1.5769
Checking accuracy on val set
Got 431 / 1000 correct (43.10)

Iteration 600, loss = 1.5903
Checking accuracy on val set
Got 484 / 1000 correct (48.40)

Iteration 700, loss = 1.3538
Checking accuracy on val set
Got 479 / 1000 correct (47.90)



In [ ]:
from torchensemble import BaggingClassifier

ensemble = BaggingClassifier(
    estimator= model,
    n_estimators = 2,
    cuda = True,
)

ensemble.set_optimizer(
    "Adam",                              
    lr=0.001,            
)

ensemble.fit(
    loader_train,
    epochs=50                      
)


Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 1.78518 | Correct: 13/41
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 1.48386 | Correct: 22/46
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 1.32180 | Correct: 25/44
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 1.36293 | Correct: 24/44
Estimator: 000 | Epoch: 000 | Batch: 400 | Loss: 1.09133 | Correct: 23/37
Estimator: 000 | Epoch: 000 | Batch: 500 | Loss: 1.39285 | Correct: 22/40
Estimator: 000 | Epoch: 000 | Batch: 600 | Loss: 1.21535 | Correct: 22/40
Estimator: 000 | Epoch: 000 | Batch: 700 | Loss: 1.25752 | Correct: 17/36
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 1.50366 | Correct: 18/40
Estimator: 001 | Epoch: 000 | Batch: 100 | Loss: 1.31512 | Correct: 21/45
Estimator: 001 | Epoch: 000 | Batch: 200 | Loss: 1.23700 | Correct: 19/41
Estimator: 001 | Epoch: 000 | Batch: 300 | Loss: 1.24639 | Correct: 21/43
Estimator: 001 | Epoch: 000 | Batch: 400 | Loss: 1.12923 | Correct: 21/37
Estimator: 001 | Epoch: 000 | Batch: 5

In [ ]:
acc = ensemble.evaluate(loader_test)
print(acc)

85.38
